In [1]:
#!pip install pulp

from pulp import *


# Solving one Branch

### 1. Defininig Model Name

In [2]:
prob = LpProblem("Branch Resource Utilization Probelm Problem", LpMaximize)

C:\Users\Vahid\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### 2. Defining Decision Variables

In [3]:
x1 = LpVariable("input1",lowBound=0.0001, upBound=None, cat = 'Continuous')
x2 = LpVariable("input2",lowBound=0.0001, upBound=None, cat = 'Continuous')
x3 = LpVariable("output",lowBound=0.0001, upBound=None, cat = 'Continuous')


### 3. Adding Objective Function

In [4]:
prob += 1000 * x3, "Utilization of Branch 1"

### 4. Adding Constarints 

In [5]:
prob += 1000*x3 - 20*x1 - 300*x2 <=0 , "B1"
prob += 1000*x3 - 30*x1 - 200*x2 <=0 , "B2"
prob += 1000*x3 - 40*x1 - 100*x2 <=0 , "B3"
prob += 1000*x3 - 20*x1 - 200*x2 <=0 , "B4"
prob += 1000*x3 - 10*x1 - 400*x2 <=0 , "B5"



prob += 20 * x1 + 300 * x2   == 1, "Normalization"


### 5. Create LP Problem

In [6]:
prob.writeLP("DEA_Branch_Model.lp")

[input1, input2, output]

### 6. Solve Problem

In [7]:
prob.solve()

1

### 7. Optimal Solution

In [8]:
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
    
print("Optimal Utilization of Branch = ", value(prob.objective))    

input1 = 0.028571429
input2 = 0.0014285714
output = 0.00085714286
Optimal Utilization of Branch =  0.85714286


### 8. Dual Prices and Slack of Surplus

In [9]:
for name, c in list(prob.constraints.items()):
    print(name, ":", c, "\t Dual:", c.pi, "\t Slack:", c.slack)

B1 : -20*input1 - 300*input2 + 1000*output <= 0 	 Dual: -0.0 	 Slack: 0.14285714
B2 : -30*input1 - 200*input2 + 1000*output <= 0 	 Dual: -0.0 	 Slack: 0.28571429
B3 : -40*input1 - 100*input2 + 1000*output <= 0 	 Dual: -0.0 	 Slack: 0.42857143
B4 : -20*input1 - 200*input2 + 1000*output <= 0 	 Dual: 0.71428571 	 Slack: -0.0
B5 : -10*input1 - 400*input2 + 1000*output <= 0 	 Dual: 0.28571429 	 Slack: -0.0
Normalization : 20*input1 + 300*input2 = 1 	 Dual: 0.85714286 	 Slack: -0.0


# Solving All Branches Together 

In [16]:
from pulp import *

performance=dict()

# decision variables
output_Ingredients=['x3']
input_Ingredients=['x1','x2']


output_vars = LpVariable.dicts("O", output_Ingredients, 0)
input_vars = LpVariable.dicts("I", input_Ingredients, 0)
inputs = {
        'B1': {'x1': 20, 'x2': 300 , 'x3': 1000},
        'B2': {'x1': 30, 'x2': 200 , 'x3': 1000},
        'B3': {'x1': 40, 'x2': 100, 'x3': 1000},
        'B4': {'x1': 20, 'x2': 200, 'x3': 1000},
        'B5': {'x1': 10, 'x2': 400, 'x3': 1000},

    }

for host in ('B1','B2','B3','B4','B5'):
    prob = LpProblem("Branch Utilization Probelm Problem", LpMaximize)

    #Objective 
    prob += (
        lpSum([inputs[host][i] * output_vars[i] for i in output_Ingredients]),
        "Utilization of Branch 1",
    )


    for h in ('B1','B2','B3','B4','B5'):
        prob += (
            lpSum([inputs[h][i] * output_vars[i] for i in output_Ingredients]+[-inputs[h][i] * input_vars[i] for i in input_Ingredients ]) <= 0,
            "{}".format(h)
        )

    #Constraints
    prob += lpSum([inputs[host][i] * input_vars[i] for i in input_Ingredients]) == 1, "Norm"        

    print("_____________________LP- Branch: {}_____________________".format(host))  
    print(prob)    

    prob.writeLP("DEA_Branch.{}_Model.lp".format(h))


    print("______________________Solution_________________________")

    prob.solve()
    #print("Status:", LpStatus[prob.status])


    for v in prob.variables():
        print(v.name, "=", v.varValue)



    print("Optimal Utilization of Branch {} = ".format(host), value(prob.objective))

    print("__________________________ Dual Variables _______________")

    for name, c in list(prob.constraints.items()):
        print(name, ":", "\t Dual:", c.pi  )
        
    performance[host]= value(prob.objective) 


_____________________LP- Branch: B1_____________________
Branch_Utilization_Probelm_Problem:
MAXIMIZE
1000*O_x3 + 0
SUBJECT TO
B1: - 20 I_x1 - 300 I_x2 + 1000 O_x3 <= 0

B2: - 30 I_x1 - 200 I_x2 + 1000 O_x3 <= 0

B3: - 40 I_x1 - 100 I_x2 + 1000 O_x3 <= 0

B4: - 20 I_x1 - 200 I_x2 + 1000 O_x3 <= 0

B5: - 10 I_x1 - 400 I_x2 + 1000 O_x3 <= 0

B6: - 1000 I_x1 - 1000 I_x2 + 1000 O_x3 <= 0

Norm: 20 I_x1 + 300 I_x2 = 1

VARIABLES
I_x1 Continuous
I_x2 Continuous
O_x3 Continuous

______________________Solution_________________________
I_x1 = 0.028571429
I_x2 = 0.0014285714
O_x3 = 0.00085714286
Optimal Utilization of Branch B1 =  0.85714286
__________________________ Dual Variables _______________
B1 : 	 Dual: -0.0
B2 : 	 Dual: -0.0
B3 : 	 Dual: -0.0
B4 : 	 Dual: 0.71428571
B5 : 	 Dual: 0.28571429
B6 : 	 Dual: -0.0
Norm : 	 Dual: 0.85714286
_____________________LP- Branch: B2_____________________
Branch_Utilization_Probelm_Problem:
MAXIMIZE
1000*O_x3 + 0
SUBJECT TO
B1: - 20 I_x1 - 300 I_x2 + 10

In [15]:
# Sorting garages in descending efficiency number
sorted_performance = {k: v for k, v in sorted(performance.items(), key=lambda item: item[1], reverse = True)}

efficient = []
inefficient = []

for h in sorted_performance.keys():
    if sorted_performance[h] >= 0.9999999:
        efficient.append(h) 
    if sorted_performance[h] < 0.9999999:
        inefficient.append(h) 
        
print('____________________________________________')
print(f"The efficient DMUs are:")
for eff in efficient:
    print(f"The performance value of DMU {eff} is: {round(performance[eff],4)}") 
    
print('____________________________________________')
print(f"The inefficient DMUs are:")
for ine in inefficient:
    print(f"The performance value of DMU {ine} is: {round(performance[ine],5)}") 

____________________________________________
The efficient DMUs are:
The performance value of DMU B3 is: 1.0
The performance value of DMU B4 is: 1.0
The performance value of DMU B5 is: 1.0
The performance value of DMU B6 is: 1.0
____________________________________________
The inefficient DMUs are:
The performance value of DMU B1 is: 0.85714
The performance value of DMU B2 is: 0.85714
